Importar librerias

In [1]:
import pandas as pd
import requests
import io
import re
from datetime import datetime
from geopy.geocoders import Nominatim

Lectura del archivo YELP bussines.pkl

In [2]:
def cargar_dataset_pickle(file_url):
    response = requests.get(file_url)
    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
    # Cargar el contenido en un DataFrame de Pandas
        df = pd.read_pickle(io.BytesIO(response.content))
        return df
    else:
        print(f'Error al obtener el archivo. Código de estado: {response.status_code}')

url='https://drive.usercontent.google.com/download?id=1byFtzpZXopdCN-XYmMHMpZqzgAqfQBBu&export=download&authuser=0&confirm=t&uuid=189d48ef-ef64-4bc9-aec7-9e654cb4c757&at=APZUnTVmlezTicVw58BDGG9siM6Q:1704808512976'
dfbusinessYelp=cargar_dataset_pickle(url)

El alcance de nuestro proyecto se limita al negocio de ulta beauty , por lo que se va a filtrar

In [3]:
#Quitamos las columnas duplicadas
dfbusinessYelp = dfbusinessYelp.loc[:, ~dfbusinessYelp.columns.duplicated()]
#filtramos ulta_beauty 
dfbusinessYelp = dfbusinessYelp[dfbusinessYelp['name'] == 'Ulta Beauty']

Borramos la columna is_open ya que no es relevante para el proyecto 

In [4]:
dfbusinessYelp = dfbusinessYelp.drop('is_open', axis=1)

La dirección que se va a utilizar es la de latitude y longitude por lo que también borramos la columna address

In [5]:
dfbusinessYelp = dfbusinessYelp.drop('address', axis=1)

La columna de name también se va a eliminar ya que todos los locales se llaman igual y solo se van a dejar los bussiness_id para identificarlos

In [6]:
dfbusinessYelp = dfbusinessYelp.drop('name', axis=1)

Borramos la columna de categories ya que todos los lugares de ultabeauty estan relacionados con el area de la belleza y esta columna es irrelevante

In [7]:
dfbusinessYelp = dfbusinessYelp.drop('categories', axis=1)

Borramos la columna de attributes y de hours

In [8]:
dfbusinessYelp = dfbusinessYelp.drop('attributes', axis=1)
dfbusinessYelp = dfbusinessYelp.drop('hours', axis=1)

Valores nulos

In [9]:
round(dfbusinessYelp.isnull().sum() / dfbusinessYelp.shape[0] * 100, 2).astype(str) + ' %'


business_id     0.0 %
city            0.0 %
state           0.0 %
postal_code     0.0 %
latitude        0.0 %
longitude       0.0 %
stars           0.0 %
review_count    0.0 %
dtype: object

Normalización del nombre de las columnas

Todos los nombres de las columnas deben estar en minusculas y las palabras deben estar separadas con '_' ya que estaremos utilizando el formato de snake_case

In [10]:
# Función para convertir el nombre de columna a formato snake_case
def snake_case(column_name):
    return re.sub(r'(?<=[a-z])(?=[A-Z])', '_', column_name).lower()

# Aplicar la función a todos los nombres de columnas
dfbusinessYelp.columns = dfbusinessYelp.columns.map(snake_case)

Nomralización de los datos dentro de las filas

Para la columna city convertiremos a snake_case

In [11]:
dfbusinessYelp['city'] = dfbusinessYelp['city'].str.lower().str.replace(' ', '_')

Para la columna state , se va a reemplazar las siglas por el nombre de los estados

In [12]:
estado_mapping = {
    'PA': 'pennsylvania',
    'FL': 'florida',
    'NV': 'nevada',
    'LA': 'louisiana',
    'AZ': 'arizona',
    'IN': 'indiana',
    'TN': 'tennessee',
    'MO': 'missouri',
    'CA': 'california',
    'ID': 'idaho',
    'NJ': 'new_jersey',
    'DE': 'delaware',
    'IL': 'illinois',
    'AB':'AB'
}

# Aplicar el mapeo a la columna 'state'
dfbusinessYelp['state'] = dfbusinessYelp['state'].map(estado_mapping)


Las review con el estado AB estan mal identificadas por lo que se usara la latitud y la longitud para determinar al estado al que pertenecen

In [13]:
def obtener_estado(latitud, longitud):
    geolocator = Nominatim(user_agent="my_geocoder")
    location = geolocator.reverse((latitud, longitud), language="en")

    if location is not None:
        # La información sobre el estado generalmente se encuentra en el nivel de "address" en la respuesta
        estado = location.raw.get('address', {}).get('state', None)
        return estado
    else:
        return None
sindato=0
faltantes=dfbusinessYelp[dfbusinessYelp.state=='AB'][['latitude', 'longitude']].drop_duplicates()
for a in range(len(faltantes)):
    latitud=faltantes.iloc[a,0]
    longitud=faltantes.iloc[a,1]

    estado = obtener_estado(latitud, longitud)

    if estado:
        dfbusinessYelp.loc[(dfbusinessYelp.latitude==latitud)&(dfbusinessYelp.longitude==longitud),'state']=estado
    else:
        sindato+=1
        continue

Agregamos la columna source porque despues se va a concatenar con el dataset de google y para poder identificar cual es la fuente de nuestros datos

In [14]:
dfbusinessYelp['source']='Y'

In [15]:
dfbusinessYelp.head()

,business_id,city,state,postal_code,latitude,longitude,stars,review_count,source
883,4uqRhXZTOzKF2ZhxbWzxfA,newark,pennsylvania,19702,39.672058,-75.6489,3.5,11,Y
1063,fWMPbickerGWohPy2vDL5A,plainfield,arizona,46168,39.713441,-86.357947,3.0,14,Y
5488,DJZQCN0NUej_EtviN4rUlg,philadelphia,pennsylvania,19131,39.978981,-75.27146,3.5,12,Y
13384,Vxqa8u_5RD5e7oBqdaU0yQ,fairview_heights,Illinois,62208,38.596645,-89.987348,3.5,13,Y
13760,idP674ti6a8yg8z2xFcCgA,newtown_square,arizona,19073,39.987189,-75.403201,2.5,24,Y
